In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import globalParameters
from gurobipy import *
from Environments.chargeEnvironment import chargeEnv
import matplotlib.pyplot as plt
from Environments.chargeSetParams import charge_SetParams

In [2]:
# Definicion de los parametros del entorno gurobi

#Dado el número de variables de este modelo, es necesario utilizar gurobi con una licencia académica (se obtiene gratuitamente en la web de gurobi)
params = globalParameters.params
env = Env(params = params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2489204
Academic license 2489204 - for non-commercial use only - registered to 80___@unizar.es


In [3]:
# Definición de los parámetros del problema
BOM, MixedItems, PurchaseItems, RouteItems, Orders, Stock, Tenv = chargeEnv(mode = "TOY")

Available_Stock = True
if not Available_Stock:
    Stock["STOCK"] = 0

Param_MOQ = True
leadtime_routes = False
leadtime_purchase = True
Param_I_0 = True
Costes_invent = True
Invent_Capacity = True
Fabrica_Capacity = True

NN, K1, K2, K3, LEVEL0, N, N_reverse, layers, R, T, D, B, item_indices, customer_indices, c_act, c1, c2, c_invent, Q_invent, Q_fabrica, MOQ1, MOQ2, lt, ltf, I_0, alpha = charge_SetParams(BOM, MixedItems, PurchaseItems, 
                                                                                                                                                                RouteItems, Orders, Stock, Tenv, Param_MOQ = Param_MOQ, leadtime_purchase = leadtime_purchase, leadtime_routes = leadtime_routes)

if not Costes_invent:
    for i in NN:
        c_invent[i] = 0
if not Invent_Capacity:
    for i in NN:
        Q_invent[i] = 50000
if not Fabrica_Capacity:
    for i in K1+K3:
        Q_fabrica[i] = 50000
    

In [4]:
# Inicialización del modelo
modelo = Model("Ejercicio", env = env)

In [5]:
# Definición de las variables
indices_x = [(i,t) for t in range(1,len(T)) for i in K1+K3]
indices_z1 = [(i,t) for t in range(1,len(T))  for i in K1+K3]
indices_y = [(i,t) for t in range(1,len(T))  for i in K2+K3]
indices_z2 = [(i,t) for t in range(1,len(T))  for i in K2+K3]
indices_w = [(i, r, t) for t in range(1,len(T))  for i in LEVEL0 for r in R]
indices_I = [(i,t) for t in range(1,len(T))  for i in NN]

x = modelo.addVars(indices_x, lb = 0, vtype = GRB.INTEGER, name = "x") 
z1 = modelo.addVars(indices_z1, lb = 0, vtype = GRB.BINARY, name = "z1")
y = modelo.addVars(indices_y, lb = 0, vtype = GRB.INTEGER, name = "y")
z2 = modelo.addVars(indices_z2, lb = 0, vtype = GRB.BINARY, name = "z2")
w = modelo.addVars(indices_w, lb = 0, vtype= GRB.BINARY, name = "w") 
It =  modelo.addVars(indices_I, lb = 0, vtype= GRB.INTEGER, name = "It")

if not Param_MOQ:
    MOQ1_indices = [i for i in K1+K3]
    MOQ1 = modelo.addVars(MOQ1_indices, lb = 1, vtype= GRB.INTEGER, name = "MOQ1") 

if not Param_I_0:
    I_0_indices = [i for i in NN]
    I_0 = modelo.addVars(I_0_indices, lb = 0, vtype= GRB.INTEGER, name = "I_0") 
    

modelo.update()

In [6]:
# Inventario para el primer periodo, para items a nivel 0
for i in set(K1).intersection(set(LEVEL0)):
    if ltf[i] < 1:
        modelo.addConstr(
            It[i,1] == I_0[i] + x[i, 1-ltf[i]] - quicksum(D[1][item_indices[i],customer_indices[r]]*w[i,r,1] for r in R),
            name=f"R10a_{i}"
        )
    if ltf[i] >= 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
            name=f"R10b_{i}"
        )
for i in set(K2).intersection(set(LEVEL0)):
    if lt[i] < 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + y[i, 1-lt[i]] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
            name=f"R20_{i}"
        )
    if lt[i] >= 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
            name=f"R30_{i}"
        )   
for i in set(K3).intersection(set(LEVEL0)):
    if lt[i] < 1:
        if ltf[i]<1:
            modelo.addConstr(
                It[i, 1] == I_0[i] + x[i, 1-ltf[i]] + y[i, 1-lt[i]] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
                name=f"R401_{i}"
            )
        else:
            modelo.addConstr(
                It[i, 1] == I_0[i] + y[i, 1-lt[i]] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
                name=f"R402_{i}"
            )
    else:
        if ltf[i]<1:            
            modelo.addConstr(
                It[i, 1] == I_0[i] + x[i, 1-ltf[i]] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
                name=f"R501_{i}"
            )
        else:
            modelo.addConstr(
                It[i, 1] == I_0[i] - quicksum(D[1][item_indices[i], customer_indices[r]] * w[i, r, 1] for r in R),
                name=f"R502_{i}"
            )
        
    

# Inventario para el primer periodo, para items a otro nivel
for i in set(K1).intersection(set().union(*layers[1:])):
    if ltf[i] < 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + x[i, 1-ltf[i]] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R1a_{i}"
        )
    else:
        modelo.addConstr(
            It[i, 1] == I_0[i] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R1b_{i}"
        )
for i in set(K2).intersection(set().union(*layers[1:])):
    if lt[i] < 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] + y[i, 1-lt[i]] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R2a_{i}"
        )
    if lt[i] >= 1:
        modelo.addConstr(
            It[i, 1] == I_0[i] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
            name=f"R3a_{i}"
        )
for i in set(K3).intersection(set().union(*layers[1:])):
    if lt[i] < 1:
        if ltf[i] < 1:
            modelo.addConstr(
                It[i, 1] == I_0[i] + x[i, 1-ltf[i]] + y[i, 1-lt[i]] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
                name=f"R4a1_{i}"
            )
        else:
            modelo.addConstr(
                It[i, 1] == I_0[i] + y[i, 1-lt[i]] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
                name=f"R4a2_{i}"
            )
    else:
        if ltf[i] < 1:
            modelo.addConstr(
                It[i, 1] == I_0[i] + x[i, 1-ltf[i]] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
                name=f"R5a1_{i}"
            )
        else:
            modelo.addConstr(
                It[i, 1] == I_0[i] - quicksum(alpha[j][i] * x[j, 1] for j in N_reverse[i]),
                name=f"R5a2_{i}"
            )
    

# Inventario para el resto de periodos, para items a nivel 0
for i in set(K1).intersection(set(LEVEL0)):
    for t in range(2, len(T)):
        if ltf[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + x[i, t-ltf[i]] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R10ta_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R10tb_{i}_{t}"
            )
for i in set(K2).intersection(set(LEVEL0)):
    for t in range(2, len(T)):
        if lt[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + y[i, t-lt[i]] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R20t_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                name=f"R30t_{i}_{t}"
            )
for i in set(K3).intersection(set(LEVEL0)):
    for t in range(2, len(T)):
        if lt[i] < t:
            if ltf[i] < t:
                modelo.addConstr(
                    It[i, t] == It[i, t-1] + x[i, t-ltf[i]] + y[i, t-lt[i]] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                    name=f"R40t1_{i}"
                )
            else:
                modelo.addConstr(
                    It[i, t] == It[i, t-1] + y[i, t-lt[i]] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                    name=f"R40t2_{i}"
                )
        else:
            if ltf[i] < t:            
                modelo.addConstr(
                    It[i, t] == It[i, t-1] + x[i, t-ltf[i]] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                    name=f"R50t1_{i}"
                )
            else:
                modelo.addConstr(
                    It[i, t] == It[i, t-1] - quicksum(D[t][item_indices[i], customer_indices[r]] * w[i, r, t] for r in R),
                    name=f"R50t2_{i}"
                )

# Inventario para el resto de periodos, para items a otro nivel
for i in set(K1).intersection(set().union(*layers[1:])):
    for t in range(2, len(T)):
        if ltf[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + x[i, t-ltf[i]] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R1atb_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R1atb_{i}_{t}"
            )
for i in set(K2).intersection(set().union(*layers[1:])):
    for t in range(2, len(T)):
        if lt[i] < t:
            modelo.addConstr(
                It[i, t] == It[i, t-1] + y[i, t-lt[i]] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R2at_{i}_{t}"
            )
        else:
            modelo.addConstr(
                It[i, t] == It[i, t-1] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                name=f"R3at_{i}_{t}"
            )
for i in set(K3).intersection(set().union(*layers[1:])):
    for t in range(2, len(T)):
        if lt[i] < t:
            if ltf[i] < t:
                modelo.addConstr(
                    It[i, t] == It[i, t-1] + x[i, t-ltf[i]] + y[i, t-lt[i]] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                    name=f"R4at1_{i}_{t}"
                )
            else:
                modelo.addConstr(
                    It[i, t] == It[i, t-1] + y[i, t-lt[i]] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                    name=f"R4at2_{i}_{t}"
                )
        else:
            if ltf[i] < t:
                modelo.addConstr(
                    It[i, t] == It[i, t-1] + x[i, t-ltf[i]] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                    name=f"R5at1_{i}_{t}"
                )
            else:
                modelo.addConstr(
                    It[i, t] == It[i, t-1] - quicksum(alpha[j][i] * x[j, t] for j in N_reverse[i]),
                    name=f"R5at2_{i}_{t}"
                )


# Restricciones de MOQs

r6 = modelo.addConstrs(
    (x[i,t] + 1000*(1-z1[i,t])>=MOQ1[i] for i in K1+K3 for t in range(1, len(T))
     ),name="R6" )
r7 = modelo.addConstrs(
    (y[i,t] + 42000*(1-z2[i,t])>= MOQ2[i] for i in K2+K3 for t in range(1, len(T))
     ),name="R7" )

# Restricciones de activacion de variables binarias
r8 = modelo.addConstrs(
    (x[i,t] <= z1[i,t]*Q_fabrica[i] for i in K1+K3 for t in range(1, len(T)) # se incluye la restriccion de capacidad de fabricacion
     ),name="R8" )
r9 = modelo.addConstrs(
    (y[i,t] <= z2[i,t]*42000 for i in K2+K3 for t in range(1, len(T))
     ),name="R9" )

# Restricciones de capacidad de Inventario
if not Param_I_0:
    r10 = modelo.addConstrs (
        (I_0[i] <= Q_invent[i] for i in NN), name="R10" )

r11 = modelo.addConstrs (
    (It[i, t] <= Q_invent[i] for i in NN for t in range(1, len(T))), name="R11" )



modelo.update()

In [7]:
# Definición de la función objetivo
modelo.setObjective(quicksum(quicksum(D[t][item_indices[i],customer_indices[r]]*B[t][item_indices[i],customer_indices[r]]*w[i,r,t] for r in R for i in LEVEL0) 
                             - quicksum(c1[i]*x[i,t] for i in K1+K3)
                             - quicksum(c2[i]*y[i,t] for i in K2+K3)
                             - quicksum(c_act[i]*z1[i,t] for i in K1+K3)
                             - quicksum(c_invent[i]*It[i,t] for i in NN)
                             for t in range(1, len(T))), sense = GRB.MAXIMIZE)

In [8]:
# Optimizacion
modelo.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12650H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Academic license 2489204 - for non-commercial use only - registered to 80___@unizar.es
Optimize a model with 224 rows, 240 columns and 484 nonzeros
Model fingerprint: 0x0bab5273
Variable types: 0 continuous, 240 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [2e+00, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 4e+04]
Presolve removed 145 rows and 157 columns
Presolve time: 0.00s
Presolved: 79 rows, 83 columns, 223 nonzeros
Variable types: 0 continuous, 83 integer (37 binary)
Found heuristic solution: objective 27910.000000
Found heuristic solution: objective 52410.000000
Found heuristic solution: objective 61287.000000

Root relaxation: objective 1.190914e+05, 38 iterations

In [9]:
# for t in range(1,len(T)+1):
#     for i in K1+K3:
#         if z1[i,t].X!=0:
#             print(i,t)
#             print(x[i,t].X)
            
# for t in range(1,len(T)+1):
#     for i in K2+K3:
#         if z2[i,t].X!=0:
#             print(i,t)
#             print(y[i,t].X)

for t in range(1, len(T)):
    for i in LEVEL0:
        for r in R:
            if w[i, r, t].X != 0 and D[t][item_indices[i],customer_indices[r]] != 0:
                print(f"Item, cliente, horizonte temporal: {i, r, t}")
                print(f"Demanda {D[t][item_indices[i],customer_indices[r]]}")
                if t == 1:
                    print(f"Stock antes: {I_0[i]}")
                else:
                    print(f"Stock antes: {It[i,t-1].X}")
                print(f"Stock despues: {It[i,t].X}")
                if i in K1+K3:
                    print(f"Cantidad Producida {x[i,t].X}")
                if i in K2+K3:
                    print(f"Cantidad Comprada {y[i,t].X}")
                    if lt[i]<t:
                        print(f"Cantidad comprada previamente que llega ahora {y[i,t-lt[i]].X}")
                    if i in K2:
                        lead_time = PurchaseItems.loc[PurchaseItems["MyBOMITEMID"] == i, "LEADTIME"].values[0]
                    if i in K3:
                        lead_time = MixedItems.loc[MixedItems["MyBOMITEMID"] == i, "LEADTIME"].values[0]
                    print(f"Lead time {lead_time}")
            else:
                if D[t][item_indices[i],customer_indices[r]] != 0:
                    print(f"Item, cliente, horizonte temporal: {i, r, t}")
                    print(f"NO SE SATISFACE")
    print("------------------------------------------")

                        

Item, cliente, horizonte temporal: (1, 'a', 1)
NO SE SATISFACE
Item, cliente, horizonte temporal: (1, 'b', 1)
NO SE SATISFACE
Item, cliente, horizonte temporal: (1, 'c', 1)
NO SE SATISFACE
Item, cliente, horizonte temporal: (2, 'a', 1)
NO SE SATISFACE
Item, cliente, horizonte temporal: (2, 'd', 1)
Demanda 15.0
Stock antes: 15
Stock despues: 0.0
Cantidad Producida 0.0
Item, cliente, horizonte temporal: (3, 'e', 1)
NO SE SATISFACE
------------------------------------------
Item, cliente, horizonte temporal: (1, 'a', 2)
Demanda 15.0
Stock antes: 0.0
Stock despues: -0.0
Cantidad Producida -0.0
Cantidad Comprada 15.0
Cantidad comprada previamente que llega ahora 15.0
Lead time 1
Item, cliente, horizonte temporal: (2, 'b', 2)
NO SE SATISFACE
Item, cliente, horizonte temporal: (2, 'f', 2)
NO SE SATISFACE
------------------------------------------
Item, cliente, horizonte temporal: (1, 'b', 3)
Demanda 15.0
Stock antes: -0.0
Stock despues: -0.0
Cantidad Producida -0.0
Cantidad Comprada 50.0
Can

In [10]:
for t in range(1,len(T)):
    for i in K2+K3:
        if z2[i,t].X!=0:
            print(f"Item, horizonte temporal {i, t}")
            print(f"Cantidad comprada {y[i,t].X}")
            print(f"Lead Time {lt[i]}")
            print(f"MOQ {MOQ2[i]}")
print("------------------------------------------")

Item, horizonte temporal (10, 1)
Cantidad comprada 5320.0
Lead Time 2
MOQ 45
Item, horizonte temporal (1, 1)
Cantidad comprada 15.0
Lead Time 1
MOQ 15
Item, horizonte temporal (8, 2)
Cantidad comprada 80.0
Lead Time 1
MOQ 10
Item, horizonte temporal (11, 2)
Cantidad comprada 3480.0
Lead Time 1
MOQ 30
Item, horizonte temporal (1, 2)
Cantidad comprada 15.0
Lead Time 1
MOQ 15
Item, horizonte temporal (1, 3)
Cantidad comprada 50.0
Lead Time 1
MOQ 15
------------------------------------------


In [11]:
for t in range(1,len(T)):
    for i in K1+K3:
        if z1[i,t].X!=0:
            print(f"Item, horizonte temporal {i, t}")
            print(f"Cantidad fabricada {x[i,t].X}")
            print(f"Lead Time {ltf[i]}")
            print(f"MOQ {MOQ1[i]}")
            print(f"Capacidad {Q_fabrica[i]}")
            

Item, horizonte temporal (3, 3)
Cantidad fabricada 90.0
Lead Time 0
MOQ 75
Capacidad 200
Item, horizonte temporal (7, 3)
Cantidad fabricada 358.0
Lead Time 0
MOQ 75
Capacidad 3000


In [12]:
for i in NN:
    print(f"Item, horizonte temporal {i, 0}")
    print(f"Stock {I_0[i]}")
    print(f"Capacidad {Q_invent[i]}")
    print("------------------------------------------")
print("------------------------------------------")
for t in range(1,len(T)):
    for i in NN:
        print(f"Item, horizonte temporal {i, t}")
        print(f"Stock {It[i,t].X}")
        print(f"Capacidad {Q_invent[i]}")
        print("------------------------------------------")
    print("------------------------------------------")

Item, horizonte temporal (1, 0)
Stock 0
Capacidad 100
------------------------------------------
Item, horizonte temporal (2, 0)
Stock 15
Capacidad 100
------------------------------------------
Item, horizonte temporal (3, 0)
Stock 10
Capacidad 100
------------------------------------------
Item, horizonte temporal (4, 0)
Stock 20
Capacidad 150
------------------------------------------
Item, horizonte temporal (5, 0)
Stock 15
Capacidad 150
------------------------------------------
Item, horizonte temporal (6, 0)
Stock 10
Capacidad 150
------------------------------------------
Item, horizonte temporal (7, 0)
Stock 2
Capacidad 150
------------------------------------------
Item, horizonte temporal (8, 0)
Stock 10
Capacidad 300
------------------------------------------
Item, horizonte temporal (9, 0)
Stock 70
Capacidad 500
------------------------------------------
Item, horizonte temporal (10, 0)
Stock 50
Capacidad 500
------------------------------------------
Item, horizonte tempo

In [13]:
for t in range(1,len(T)):
    for i in K1+K3:
        if z1[i,t].X!=0:
            print(f"Item, horizonte temporal {i, t}")
            print(f"Cantidad producida {x[i,t].X}")
            print(f"MOQ {MOQ1[i]}")
        else:
            if x[i,t].X != 0:
                print(f"No debería producir {i, t}")
                print(f"Cantidad producida {x[i,t].X}")

Item, horizonte temporal (3, 3)
Cantidad producida 90.0
MOQ 75
Item, horizonte temporal (7, 3)
Cantidad producida 358.0
MOQ 75


In [14]:
NoSatisfecha = 0
totalPedidos = 0
udsNoSatisfecha = 0
totalUds = 0
for t in range(1, len(T)):
    for i in LEVEL0:
        for r in R:
            if D[t][item_indices[i],customer_indices[r]] != 0:
                if w[i, r, t].X == 0:
                    NoSatisfecha += 1
                    udsNoSatisfecha += D[t][item_indices[i],customer_indices[r]]
                totalPedidos += 1
                totalUds += D[t][item_indices[i],customer_indices[r]]


print(udsNoSatisfecha/totalUds*100)
print(NoSatisfecha/totalPedidos*100)
print(modelo.getAttr("ObjVal"))


55.172413793103445
56.25
113740.0


In [14]:
for t in range(1, len(T)):
    for i in K1+K3:
        print(f"MOQ1 {MOQ1[i].X}")
                

MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0
MOQ1 1.0


In [ ]:
modelo.close()
env.close()